# PostHoc Analysis

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.model_selection import cross_val_score, RepeatedKFold, train_test_split
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error, r2_score

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

sns.set_theme(style="whitegrid")

In [3]:
#change directory for your path
os.chdir('E:\\OneDrive\\Documents\\GitHub\\eHealthEquity\\Research\\Data Source')

In [4]:
df = pd.read_csv('final_merge.csv')

target = [
    'brfss_diabetes_ageadjprv' # percentage of adults aged >=18 years who have diagnosed diabetes
]
features = [
    'pct_dg_bb_int', # percentage of households with broadband internet access
    "brfss_access2_ageadjprv", # percentage of adults aged >=18 years who are unable to see a doctor in the past 12 months due to cost
    "brfss_checkup_ageadjprv", # percentage of adults aged >=18 years who had a checkup in the past 12 months
    "brfss_mhlth_ageadjprv", # percentage of adults aged >=18 years who report 14 or more days of poor mental health in the past 30 days
    "brfss_obesity_ageadjprv", # percentage of adults aged >=18 years who have obesity
    "pct_ed_lt9", # percentage of adults aged >=25 years with less than a 9th grade education
    "pct_ed_9_12", # percentage of adults aged >=25 years with a 9th to 12th grade education and no diploma
    "pct_ed_hs", # percentage of adults aged >=25 years with a high school diploma or GED
    "pct_ed_asc", # percentage of adults aged >=25 years with an associate's degree
    "pct_age_gte65", # percentage of adults aged >=65 years
    "pct_occ_unemp", #  percentage of employed adults aged >=16 years who are unemployed
    "pct_ses_pov",  # percentage of all people in poverty
    "pct_tp_veh_0", # percentage of households with zero vehicles
    "ruca_rural", # rural-urban continuum codes
    "bb_int_struct" # broadband internet access availability 
]
geo = [
    'state',
    'county'
]
df = df[geo + target + features]

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3076 entries, 0 to 3075
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   state                     3076 non-null   object 
 1   county                    3076 non-null   object 
 2   brfss_diabetes_ageadjprv  3076 non-null   float64
 3   pct_dg_bb_int             3076 non-null   float64
 4   brfss_access2_ageadjprv   3076 non-null   float64
 5   brfss_checkup_ageadjprv   3076 non-null   float64
 6   brfss_mhlth_ageadjprv     3076 non-null   float64
 7   brfss_obesity_ageadjprv   3076 non-null   float64
 8   pct_ed_lt9                3076 non-null   float64
 9   pct_ed_9_12               3076 non-null   float64
 10  pct_ed_hs                 3076 non-null   float64
 11  pct_ed_asc                3076 non-null   float64
 12  pct_age_gte65             3076 non-null   float64
 13  pct_occ_unemp             3076 non-null   float64
 14  pct_ses_

# Predictive Power

In [5]:
X = df[features]
y = df[target]
y = np.array(y).ravel()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=418)

lr = LinearRegression(fit_intercept=True)
lr.fit(X_train, y_train)
print('Linear Regression')
print('Size of training set: ', X_train.shape[0])
print('Size of testing set: ', X_test.shape[0])

# Assess MAE, MSE, RMSE, and R^2 with cross validation
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=418)

print('')
mae = cross_val_score(lr, X_train, y_train, scoring=make_scorer(mean_absolute_error), cv=cv, n_jobs=-1)
print('MAE: %.3f (%.3f)' % (np.mean(mae), np.std(mae)))
mse = cross_val_score(lr, X_train, y_train, scoring=make_scorer(mean_squared_error), cv=cv, n_jobs=-1)
print('MSE: %.3f (%.3f)' % (np.mean(mse), np.std(mse)))
rmse = cross_val_score(lr, X_train, y_train, scoring=make_scorer(mean_squared_error, squared=False), cv=cv, n_jobs=-1)
print('RMSE: %.3f (%.3f)' % (np.mean(rmse), np.std(rmse)))
r2 = cross_val_score(lr, X_train, y_train, scoring=make_scorer(r2_score), cv=cv, n_jobs=-1)
print('R^2: %.3f (%.3f)' % (np.mean(r2), np.std(r2)))

# Assess MAE, MSE, RMSE, and R^2 with test set
y_pred = lr.predict(X_test)
print('')
print('Test set')
mae = mean_absolute_error(y_test, y_pred)
print('MAE: %.3f' % mae)
mse = mean_squared_error(y_test, y_pred)
print('MSE: %.3f' % mse)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print('RMSE: %.3f' % rmse)
r2 = r2_score(y_test, y_pred)
print('R^2: %.3f' % r2)


Linear Regression
Size of training set:  2307
Size of testing set:  769

MAE: 0.533 (0.032)
MSE: 0.482 (0.065)
RMSE: 0.693 (0.046)
R^2: 0.905 (0.015)

Test set
MAE: 0.558
MSE: 0.520
RMSE: 0.721
R^2: 0.894


# Geographic Analysis
Organized by larger regions defined by the US Census Bureau. The regions are Northeast, Midwest, South, and West.

In [6]:
regions = {
    'Northeast': ['Connecticut', 'Maine', 'Massachusetts', 'New Hampshire', 'Rhode Island', 'Vermont', 'New Jersey', 'New York', 'Pennsylvania'],
    'Midwest': ['Indiana', 'Illinois', 'Michigan', 'Ohio', 'Wisconsin', 'Iowa', 'Kansas', 'Minnesota', 'Missouri', 'Nebraska', 'North Dakota', 'South Dakota'],
    'South': ['Delaware', 'Georgia', 'Maryland', 'North Carolina', 'South Carolina', 'Virginia', 'District of Columbia', 'West Virginia', 'Alabama', 'Kentucky', 'Mississippi', 'Tennessee', 'Arkansas', 'Louisiana', 'Oklahoma', 'Texas'],
    'West': ['Arizona', 'Colorado', 'Idaho', 'Montana', 'Nevada', 'New Mexico', 'Utah', 'Wyoming', 'Alaska', 'California', 'Hawaii', 'Oregon', 'Washington']
}

df['region'] = df['state'].apply(lambda x: [k for k, v in regions.items() if x in v][0])

unmapped = df[~df['region'].isin(regions.keys())]
unmapped['state'].unique()

array([], dtype=object)

In [7]:
def regression_by_region(region_name, data, predictors, target):
    region_data = data[data['region'] == region_name]
    region_target = region_data[target]
    region_features = region_data[predictors]
    region_features = sm.add_constant(region_features)
    region_model = sm.OLS(region_target, region_features).fit()
    bb_int_coef = region_model.params['pct_dg_bb_int']

    return region_model, bb_int_coef

region_coef = {}

for region in df['region'].unique():
    region_model, bb_int_coef = regression_by_region(region, df, features, target)
    region_coef[region] = bb_int_coef

region_coef

{'West': -0.025263398782000568,
 'South': -0.01828708703566905,
 'Northeast': -0.004039884588030501,
 'Midwest': -0.0060562331377255465}

In [8]:
def regression_by_state(state_name, data, predictors, target):
    state_data = data[data['state'] == state_name]
    state_target = state_data[target]
    state_features = state_data[predictors]
    state_features = sm.add_constant(state_features)
    state_model = sm.OLS(state_target, state_features).fit()
    bb_int_coef = state_model.params['pct_dg_bb_int']

    return state_model, bb_int_coef

state_coef = {}

for state in df['state'].unique():
    state_model, bb_int_coef = regression_by_state(state, df, features, target)
    state_coef[state] = bb_int_coef

state_coef


{'Alaska': 0.003337652597036861,
 'Alabama': 0.004319208323070034,
 'Arkansas': -0.013917857862492774,
 'Arizona': -0.026084625363885006,
 'California': 0.007077215421293609,
 'Colorado': 0.003661770422812516,
 'Connecticut': -0.040008657779590284,
 'District of Columbia': 0.0477302272542152,
 'Delaware': 0.0384517170165484,
 'Georgia': -0.015486746955361976,
 'Hawaii': 0.028389613329861764,
 'Iowa': 0.003440177000497091,
 'Idaho': -0.02174183641102348,
 'Illinois': -0.022505141692996868,
 'Indiana': -0.0005259810451446745,
 'Kansas': 0.007282223979073067,
 'Kentucky': 0.0008248140668348086,
 'Louisiana': -0.02838687000113499,
 'Massachusetts': -0.020600390096106647,
 'Maryland': -0.04344567206621641,
 'Maine': -0.08745449491279667,
 'Michigan': -0.011120985636572295,
 'Minnesota': 0.0001722426791944398,
 'Missouri': 0.011724826594330294,
 'Mississippi': -0.0051492125271358005,
 'Montana': 0.0029040539791703596,
 'North Carolina': -0.023907360215902493,
 'North Dakota': -0.009215295970

# Machine Learning : Alternative Models

In [9]:
#Alternative ML models with GridSearchCV

# models = {
#     'Decision Tree': DecisionTreeRegressor(),
#     'Random Forest': RandomForestRegressor(),
#     'Gradient Boosting': GradientBoostingRegressor(),
#     'K-Nearest Neighbors': KNeighborsRegressor(),
#     'Multi-Layer Perceptron': MLPRegressor()
# }


# dt_params = {
#     'max_depth': [2, 3, 4, 5, 6, 7, 8, 9, 10],
#     'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10],
#     'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9 ,10]
# }

# rf_params = {
#     'n_estimators': [50, 150],
#     'max_depth': [2, 5, 10],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 6, 10]
# }

# gb_params = {
#     'n_estimators': [50, 150],
#     'max_depth': [2, 5, 10],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 6, 10],
#     'learning_rate': [0.01, 0.1]
# }

# knn_params = {
#     'n_neighbors': [3, 5, 7],
#     'weights': ['uniform', 'distance'],
#     'p': [1, 2]
# }

# mlp_params = {
#     'hidden_layer_sizes': [(10,), (20,), (30,), (40,), (50,)],
#     'activation': ['relu'],
#     'solver': ['adam'],
#     'learning_rate': ['constant', 'adaptive']
# }

# params = {
#     'Decision Tree': dt_params,
#     'Random Forest': rf_params,
#     'Gradient Boosting': gb_params,
#     'K-Nearest Neighbors': knn_params,
#     'Multi-Layer Perceptron': mlp_params
# }


In [10]:
# for model_name, model in models.items():
#     print('Model:', model_name)
#     grid_search = GridSearchCV(model, params[model_name], cv=15, scoring='neg_mean_absolute_error', verbose=1, n_jobs=-1)
#     grid_search.fit(x, y)
#     print('Best Parameters:', grid_search.best_params_)
#     print('Best Score:', -grid_search.best_score_)
#     print()

Model: Decision Tree
Fitting 15 folds for each of 810 candidates, totalling 12150 fits
Best Parameters: {'max_depth': 10, 'min_samples_leaf': 9, 'min_samples_split': 2}
Best Score: 0.6734945711092055

Model: Random Forest
Fitting 15 folds for each of 72 candidates, totalling 1080 fits
Best Parameters: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 150}
Best Score: 0.5262649520665891

Model: Gradient Boosting
Fitting 15 folds for each of 144 candidates, totalling 2160 fits
Best Parameters: {'learning_rate': 0.1, 'max_depth': 2, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 150}
Best Score: 0.5097996372360977

Model: K-Nearest Neighbors
Fitting 15 folds for each of 12 candidates, totalling 180 fits
Best Parameters: {'n_neighbors': 7, 'p': 1, 'weights': 'distance'}
Best Score: 0.5438632275828476

Model: Multi-Layer Perceptron
Fitting 15 folds for each of 10 candidates, totalling 150 fits
Best Parameters: {'activation': 'relu', 'hidden_layer_sizes': (40,), 'learning_rate': 'adaptive', 'solver': 'adam'}
Best Score: 0.48095450827868647

In [11]:
# RandomSearchCV

# from sklearn.model_selection import RandomizedSearchCV

# for model_name, model in models.items():
#     print('Model:', model_name)
#     random_search = RandomizedSearchCV(model, params[model_name], cv=15, scoring='neg_mean_absolute_error', verbose=1, n_jobs=-1)
#     random_search.fit(x, y)
#     print('Best Parameters:', random_search.best_params_)
#     print('Best Score:', -random_search.best_score_)
#     print()

Model: Linear Regression
Fitting 15 folds for each of 1 candidates, totalling 15 fits
Best Parameters: {'fit_intercept': True}
Best Score: 0.6034274009228865

Model: Decision Tree
Fitting 15 folds for each of 10 candidates, totalling 150 fits
Best Parameters: {'min_samples_split': 8, 'min_samples_leaf': 5, 'max_depth': 10}
Best Score: 0.6823770482656772

Model: Random Forest
Fitting 15 folds for each of 10 candidates, totalling 150 fits
Best Parameters: {'n_estimators': 150, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 10}
Best Score: 0.5280400874120184

Model: Gradient Boosting
Fitting 15 folds for each of 10 candidates, totalling 150 fits
Best Parameters: {'n_estimators': 150, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_depth': 2, 'learning_rate': 0.1}
Best Score: 0.5110764727785839

Model: K-Nearest Neighbors
Fitting 15 folds for each of 10 candidates, totalling 150 fits
Best Parameters: {'weights': 'distance', 'p': 1, 'n_neighbors': 7}
Best Score: 0.5438632275828476

Model: Multi-Layer Perceptron
Fitting 15 folds for each of 10 candidates, totalling 150 fits
Best Parameters: {'solver': 'adam', 'learning_rate': 'adaptive', 'hidden_layer_sizes': (40,), 'activation': 'relu'}
Best Score: 0.47733495977365


In [12]:
#additional MLP parameters 
mlp_params = {
    'hidden_layer_sizes': [(80,40,5), (40,20,10), (40,), (40,10,4)],
    'activation': ['relu'],
    'solver': ['adam'],
    'learning_rate': ['constant', 'adaptive'],
    'alpha': [0.0001, 0.001, 0.01, 0.1],
    'early_stopping': [True]
}

mlp = MLPRegressor()

mlp_random_search = RandomizedSearchCV(mlp, mlp_params, cv=15, scoring='neg_mean_absolute_error', verbose=1, n_jobs=-1)
mlp_random_search.fit(X, y)
print('Best Parameters:', mlp_random_search.best_params_)
print('Best Score:', -mlp_random_search.best_score_)
print()



Fitting 15 folds for each of 10 candidates, totalling 150 fits


Best Parameters: {'solver': 'adam', 'learning_rate': 'adaptive', 'hidden_layer_sizes': (40, 10, 4), 'early_stopping': True, 'alpha': 0.1, 'activation': 'relu'}
Best Score: 0.5380674037145875



# MLP Predictive Capabilities and Cross-Fold Validation

In [13]:
#best model from hyperparameter tuning
model = MLPRegressor(activation='relu', hidden_layer_sizes=(40, 10, 4), learning_rate='constant', solver='adam', max_iter=1000, random_state=42, early_stopping=True, alpha=0.0001)
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)
cv_r2 = cross_val_score(model, X, y, cv=cv, scoring='r2', n_jobs=-1)
cv_rmse = cross_val_score(model, X, y, cv=cv, scoring='neg_root_mean_squared_error', n_jobs=-1)
cv_abs_error = cross_val_score(model, X, y, cv=cv, scoring='neg_mean_absolute_error', n_jobs=-1)

print('R2 Mean:', cv_r2.mean())
print('RMSE Mean:', -cv_rmse.mean())
print('MAE Mean:', -cv_abs_error.mean())


R2 Mean: 0.9125859466652979
RMSE Mean: 0.6634980851044062
MAE Mean: 0.5093113053207143


# OLS Predictive Capabilities and Cross-fold validation

In [14]:
#linear model
model = LinearRegression(fit_intercept=True)
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)
cv_r2 = cross_val_score(model, X, y, cv=cv, scoring='r2', n_jobs=-1)
cv_rmse = cross_val_score(model, X, y, cv=cv, scoring='neg_root_mean_squared_error', n_jobs=-1)
cv_abs_error = cross_val_score(model, X, y, cv=cv, scoring='neg_mean_absolute_error', n_jobs=-1)

print('R2 Mean:', cv_r2.mean())
print('RMSE Mean:', -cv_rmse.mean())
print('MAE Mean:', -cv_abs_error.mean())


R2 Mean: 0.9029289519927749
RMSE Mean: 0.6998194188651131
MAE Mean: 0.5387643644848027


In [19]:
# import optuna
# from optuna import Trial, visualization

# def objective(trial: Trial) -> float:
#     #MLP parameters
#     activation = trial.suggest_categorical('activation', ['relu'])
#     solver = trial.suggest_categorical('solver', ['adam'])
#     learning_rate = trial.suggest_categorical('learning_rate', ['constant', 'adaptive'])
#     alpha = trial.suggest_loguniform('alpha', 1e-4, 1e-1)
#     early_stopping = trial.suggest_categorical('early_stopping', [True])
#      # Hidden layer sizes
#     n_layers = trial.suggest_int('n_layers', 1, 6)

#     # Hidden layer sizes
#     hidden_layer_sizes = tuple(
#         trial.suggest_int(f'hidden_layer_{i}', 1, 100) for i in range(n_layers)
#     )
    
#     # Create model
#     model = MLPRegressor(activation=activation, hidden_layer_sizes=hidden_layer_sizes, learning_rate=learning_rate, solver=solver, max_iter=1000, random_state=42, early_stopping=early_stopping, alpha=alpha)
    
#     # Cross-validation
#     cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)
#     cv_abs_error = cross_val_score(model, X, y, cv=cv, scoring='neg_mean_absolute_error', n_jobs=-1)
#     return cv_abs_error.mean()


# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=100)
# print('Best Trial:', study.best_trial.params)
# print('Best Score:', study.best_value)

# #visualize the optimization history
# optuna.visualization.plot_optimization_history(study)



[I 2023-12-04 08:08:46,640] A new study created in memory with name: no-name-fe0732e8-a75e-447f-9b06-79f6dd6f8be3
C:\Users\mfarm\AppData\Local\Temp\ipykernel_6324\2628198864.py:9: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.

[I 2023-12-04 08:08:56,765] Trial 0 finished with value: -0.5003002192965016 and parameters: {'activation': 'relu', 'solver': 'adam', 'learning_rate': 'constant', 'alpha': 0.004878476695470025, 'early_stopping': True, 'n_layers': 5, 'hidden_layer_0': 54, 'hidden_layer_1': 36, 'hidden_layer_2': 90, 'hidden_layer_3': 76, 'hidden_layer_4': 28}. Best is trial 0 with value: -0.5003002192965016.
[I 2023-12-04 08:09:03,697] Trial 1 finished with value: -0.5321366553108405 and parameters: {'activation': 'relu', 'solver': 'adam', 'learning_rate': 'adaptive', 'alpha': 0.004285083461875077, 'early_stopping': True, 'n

Best Trial: {'activation': 'relu', 'solver': 'adam', 'learning_rate': 'adaptive', 'alpha': 0.004760239925367831, 'early_stopping': True, 'n_layers': 6, 'hidden_layer_0': 96, 'hidden_layer_1': 100, 'hidden_layer_2': 82, 'hidden_layer_3': 67, 'hidden_layer_4': 62, 'hidden_layer_5': 90}
Best Score: -0.47999646593766615


In [21]:
mlp = MLPRegressor(activation='relu', hidden_layer_sizes=(96,100,82,67,62,90), learning_rate='adaptive',
                    solver='adam', max_iter=1000, random_state=42, early_stopping=True, alpha=0.004760239925367831)
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)
cv_r2 = cross_val_score(mlp, X, y, cv=cv, scoring='r2', n_jobs=-1)
cv_rmse = cross_val_score(mlp, X, y, cv=cv, scoring='neg_root_mean_squared_error', n_jobs=-1)
cv_abs_error = cross_val_score(mlp, X, y, cv=cv, scoring='neg_mean_absolute_error', n_jobs=-1)

print('R2 Mean:', cv_r2.mean())
print('RMSE Mean:', -cv_rmse.mean())
print('MAE Mean:', -cv_abs_error.mean())

#print params
print('Best Trial:', study.best_trial.params)

R2 Mean: 0.9227780614492392
RMSE Mean: 0.6236795230952153
MAE Mean: 0.47999646593766615
Best Trial: {'activation': 'relu', 'solver': 'adam', 'learning_rate': 'adaptive', 'alpha': 0.004760239925367831, 'early_stopping': True, 'n_layers': 6, 'hidden_layer_0': 96, 'hidden_layer_1': 100, 'hidden_layer_2': 82, 'hidden_layer_3': 67, 'hidden_layer_4': 62, 'hidden_layer_5': 90}
Best Score: -0.47999646593766615
